In [7]:
from pyspark.sql import SparkSession, SQLContext 
from pyspark.sql.functions import lower
import pyspark.sql.functions as sf ##TESTING
from operator import add
import re
from collections import OrderedDict
from operator import itemgetter 
import itertools
import json

import pyspark.sql.functions as f  #TA BORT




spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("blameyben_lecture1_simple_example")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

spark_context = spark_session.sparkContext #added
sqlContext = SQLContext(spark_context)



In [8]:
from pyspark.sql.types import *
schema = StructType([])




In [23]:
import timeit
start_time = timeit.default_timer()




#file = open('testfiles/sample_data.json')
file = open('testfiles/sample_data-Copy1.json')

content = file.read()
array = content.split('\n')

file.close()
    
#subreddit_array = ["Cubers", "mildlyinteresting","4chan4trump", "The_Donald"]

df = sqlContext.createDataFrame(spark_context.emptyRDD(), schema)


for comment in array:
    
    rddjson = spark_context.parallelize([comment])
    
    current_df = sqlContext.read.json(rddjson)
       
    columns_to_drop = ['author_cakeday','id', 'id_copy']

    if 'author_cakeday' in current_df.columns:
        current_df = current_df.drop('author_cakeday')
        
    current_df = current_df.drop('author_flair_css_class','author_flair_text','can_gild','distinguished','edited','id','is_submitter','link_id','parent_id','permalink','retrieved_on','stickied','subreddit_id')
  
    count = df.count()
    


    if count == 0:
        df = current_df
    else:
        df = df.union(current_df)


elapsed = timeit.default_timer() - start_time

#df.show()

print("Time elapsed: ",elapsed) 
    

Time elapsed:  83.33103949110955


In [24]:
start_time = timeit.default_timer()

#df.select("author").show()
df.groupBy('subreddit').count().sort("count", ascending = False).show(10)

elapsed = timeit.default_timer() - start_time

print("Time elapsed: ",elapsed)

+------------------+-----+
|         subreddit|count|
+------------------+-----+
|               CFB|    3|
|           xboxone|    1|
|  learnprogramming|    1|
|         2007scape|    1|
|               HFY|    1|
|ImagesOfNewZealand|    1|
|   fantasyfootball|    1|
|          gonewild|    1|
|          Roughsex|    1|
|      tvcrossovers|    1|
+------------------+-----+
only showing top 10 rows

Time elapsed:  3.9419189877808094
